In [1]:
import os
import pandas as pd
from valuebank import rephrase

In [2]:
os.chdir('..')

## Load in original data

In [3]:
procon_file = 'external_data/procon20/train.tsv'
df = pd.read_csv(procon_file, sep='\t', header=None, names=["pro", "issue", "argument"])
df['pro'] = df['pro'].apply(lambda x: x == 1) # '01' is pro '10' is con

In [4]:
df.columns

Index(['pro', 'issue', 'argument'], dtype='object')

In [5]:
df['issue'] = df.apply(lambda x: x['issue'].split('?')[0]+'?', axis=1)

In [6]:
issues = df['issue'].unique()

In [7]:
tiny_issues = issues[0:10]

In [12]:
answers = [['yes', 'no']] * len(tiny_issues)
questions = list(zip(tiny_issues, answers))

In [13]:
rephrase(questions, 'gpt-4-0613', count_tokens=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1465.21it/s]

Total input tokens: 1496
Total output tokens: 1280
Total cost: $0.12167999999999998


In [14]:
rephrased = rephrase(questions, 'gpt-4-0613', count_tokens=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.44s/it]


In [15]:
records = []
for issue, rephrasings in zip(tiny_issues, rephrased):
    questions = set([issue] + rephrasings)
    for question in questions:
        records.append({'question' : question, 'options' : ['yes', 'no'], 'original' : issue, 'rephrase' : question != issue})

In [51]:
filename = 'procon_tiny.jsonl'
pd.DataFrame(records).to_json(filename, lines=True, orient='records')

## old rephrasing

In [34]:
REPHRASE = "Rephrase the following question as a gerund. Do not capitalize or end the phrase with a period.\n\n"

few_shot_messages =[
    {
      "role": "user",
      "content": REPHRASE + "\"Should the Death Penalty Be Legal?\""
    },
    {
      "role": "assistant",
      "content": "legalizing the death penalty"
    },
    {
      "role": "user",
      "content": REPHRASE + "\"Is Solar Power an Economical Alternative to Conventional Energy?\""
    },
    {
      "role": "assistant",
      "content": "the economical viability of solar power as an alternative to conventional energy"
    }]

In [26]:
client = openai.OpenAI(organization ="org-EryH1QYCQUIHJGviV3ile7Z1" )

In [35]:
rephrased = []
for question in issues:
    messages = copy.deepcopy(few_shot_messages)
    messages.append({'role' : 'user', 'content' : REPHRASE + f'"{question}"'})
    result = client.chat.completions.create(model='gpt-3.5-turbo-1106',
                                               messages=(messages),
                                               temperature=0,
                                               max_tokens=30)
    rephrased.append(result.choices[0].message.content)

In [47]:
filename = 'procon.jsonl'
pd.DataFrame({'issue' : issues, 'gerund' : rephrased}).to_json(filename, lines=True, orient='records')